In [1]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import sys
import time
import pandas as pd
import cv2
import os
import imageio
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import shutil
import random

from deepcell.utils.tracking_utils import load_trks

/opt/conda/lib/python3.7/site-packages/tensorflow-2.5.0-py3.7-linux-x86_64.egg/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [3]:
# Input file
fN = h5py.File("../22956814/22956814_featuresN.hdf5")

# Define dorsal and ventral coordinates
coordinates = fN['coordinates']
dorsal = coordinates['dorsal_contours']
ventral = coordinates['ventral_contours']

# trajectory_data and timeseries_data
traj = pd.read_hdf("../22956814/22956814_featuresN.hdf5", '/trajectories_data')
timeseries = fN['timeseries_data']

In [4]:
# Get wells - data from timeseries_data, at position [2]

wells = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4']
#wells = [each.encode() for each in wells]

well_dict = {}
for each in wells:
    well_dict[each] = []
    
#print('A1'.encode().decode())
for i, each in enumerate(timeseries[:]):  # "i" == index
    well = each[2].decode()
    if well in well_dict.keys():
        well_dict[well].append(i)

In [83]:
# ---------------------------NEW---------------------------------------

import gc

def make_lineage(well, mask_npy, suffix):
    """suffix = the number at the end of the mask_npy"""
    
    array = np.load(mask_npy)
    unique_worm_ids = list(np.unique(array))[1:]
    
    txt = '../22956814/%s_not_empty.txt' % well
    with open(txt, 'r') as f:
        start_end_samplings = f.read().splitlines()  # Cool!! 
        frame_start, frame_end, down_sampling = start_end_samplings[suffix-1].split(",")
        sampling = range(int(frame_start), int(frame_end)+1, int(down_sampling))
            
    pd = traj.loc[well_dict[well], :]
    lineages = {}
    
    for worm in unique_worm_ids:
        #worm_pd = pd.loc[pd['worm_index_joined'] == worm]

        nested_dict = {}
        nested_dict['label'] = worm

        frame_list = [] 
        
        for i, frame in enumerate(array[:]):
            unique_pixels = list(np.unique(frame))
            if worm in unique_pixels:
                frame_list.append(i)
                
        nested_dict = {}
        nested_dict['label'] = worm
        nested_dict['frames'] = frame_list
        nested_dict['daughters'] = []
        nested_dict['capped'] = False
        nested_dict['frame-div'] = None
        nested_dict['parent'] = None

        lineages[worm] = nested_dict
    
    del array
    gc.collect()
        
    return lineages

In [86]:
a1_1 = make_lineage('A1', "../22956814/A1_mask_1.npy", 1) 

# Well A2
a2_1 = make_lineage('A2', "../22956814/A2_mask_1.npy", 1) 
a2_2 = make_lineage('A2', "../22956814/A2_mask_2.npy", 2) 
a2_3 = make_lineage('A2', "../22956814/A2_mask_3.npy", 3) 
a2_4 = make_lineage('A2', "../22956814/A2_mask_4.npy", 4) 
a2_5 = make_lineage('A2', "../22956814/A2_mask_5.npy", 5) 
a2_6 = make_lineage('A2', "../22956814/A2_mask_6.npy", 6) 
a2_7 = make_lineage('A2', "../22956814/A2_mask_7.npy", 7) 
a2_8 = make_lineage('A2', "../22956814/A2_mask_8.npy", 8) 
a2_9 = make_lineage('A2', "../22956814/A2_mask_9.npy", 9) 
a2_10 = make_lineage('A2', "../22956814/A2_mask_10.npy", 10) 

# Well A3
a3_1 = make_lineage('A3', "../22956814/A3_mask_1.npy", 1) 
a3_2 = make_lineage('A3', "../22956814/A3_mask_2.npy", 2) 
a3_3 = make_lineage('A3', "../22956814/A3_mask_3.npy", 3) 
a3_4 = make_lineage('A3', "../22956814/A3_mask_4.npy", 4) 
a3_5 = make_lineage('A3', "../22956814/A3_mask_5.npy", 5) 
a3_6 = make_lineage('A3', "../22956814/A3_mask_6.npy", 6) 
a3_7 = make_lineage('A3', "../22956814/A3_mask_7.npy", 7) 
a3_8 = make_lineage('A3', "../22956814/A3_mask_8.npy", 8) 
a3_9 = make_lineage('A3', "../22956814/A3_mask_9.npy", 9) 

# Well A4
a4_1 = make_lineage('A4', "../22956814/A4_mask_1.npy", 1) 
a4_2 = make_lineage('A4', "../22956814/A4_mask_2.npy", 2) 
a4_3 = make_lineage('A4', "../22956814/A4_mask_3.npy", 3) 
a4_4 = make_lineage('A4', "../22956814/A4_mask_4.npy", 4) 
a4_5 = make_lineage('A4', "../22956814/A4_mask_5.npy", 5) 

In [87]:
import psutil
psutil.virtual_memory()

svmem(total=439595151360, available=391478181888, percent=10.9, used=44452388864, free=297573847040, active=46360567808, inactive=92048850944, buffers=143462400, cached=97425453056, shared=9052160, slab=2970976256)

In [91]:
a4_

{3: {'label': 3,
  'frames': [0, 1, 2, 5, 6, 7, 8, 10, 11, 12],
  'daughters': [],
  'capped': False,
  'frame-div': None,
  'parent': None}}

In [72]:
# Check gif

def plot_gif(np_array, output_fn):
    mpl.rcParams['image.interpolation'] = 'none'  # Prevent mpl smoothes the edges
    os.mkdir("pic_temporary")  # To temporarily store the files

    #spring = cm.get_cmap('spring', 256)
    #newcolors = spring(np.linspace(0, 256, 256))
    #black = np.array([0/256, 0/256, 0/256, 1])
    #newcolors[:1, :] = black
    #newcmp = ListedColormap(newcolors)

    pic_list = []
    flag = 0

    for i in range(np_array.shape[0]):
        #plt.imshow(np_array[i], cmap=newcmp)
        mask = np_array[i]
        mask = np.where(mask==0, -1, mask) 
        #print(np.unique(mask_1))

        value = -1
        masked_array = np.ma.masked_where(mask == value, mask)

        #cmap = mpl.cm.get_cmap("spring")
        cmap = mpl.cm.get_cmap("spring").copy()
        cmap.set_bad(color='black')
        
        plt.imshow(masked_array, cmap=cmap)
        
        fn = 'pic_temporary/%d.jpg' % flag
        plt.savefig(fn)
        pic_list.append(fn)
        flag += 1
    
    with imageio.get_writer(output_fn, mode='I') as writer:
        for filename in pic_list:
            image = imageio.imread(filename)
            writer.append_data(image)
    
    # Remove the temporary file
    shutil.rmtree("pic_temporary")

In [ ]:
plot_gif(np.load('../22956814/A4_mask_4.npy'), '../22956814/gif/A4_mask_4.gif')